## **Agentes e o uso do RAG em Langchain - Tutor Inteligente**
*Por Kemilli Lima*

Utilizando agentes e RAG para gerar perguntas para o processo de preparação da POSCOMP. Essa é a versão inicial da aplicação, na qual o usuário pode anexar uma url, um pdf ou um arquivo txt para auxiliar na preparação das questões.



### **Instalação, importação de API's utilizadas ao longo da aplicação e validação da chave**

In [ ]:
!pip install langchain langchain-community langchain-openai openai pypdf faiss-cpu python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain.agents import Tool, initialize_agent, AgentType


In [ ]:
import dotenv
dotenv.load_dotenv('/content/k.env')

False

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### **Testes para se adpatar a sintaxe do Langchain**

In [ ]:
# Teste básico
def generate_animal_name(animal_type, color):
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    prompt_animal_name = PromptTemplate(
        input_variables=['animal_type', 'color'],
        template="You have an animal {animal_type} with {color} color. Suggest 5 names for this animal."
    )

    animal_name_chain = LLMChain(llm=llm, prompt=prompt_animal_name)
    output = animal_name_chain.run({'animal_type': animal_type, 'color': color})
    return output

print(generate_animal_name('cat', 'white'))


1. Snowball
2. Marshmallow
3. Cotton
4. Pearl
5. Blizzard


In [ ]:
def generate_questions(content):
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    prompt_questions = PromptTemplate(
        input_variables = ['content'],
        template = "You have a content about {content}. Suggest 5  multiple choice questions (5 options) with answers for this content in Markdown and in Portuguese "
    )

    questions_chain = LLMChain(llm=llm, prompt=prompt_questions)
    output = questions_chain.run({'content': content})
    return output

print(generate_questions('Padrões GRASP'))


1. Qual é o objetivo dos padrões GRASP?
   - A) Definir padrões de design de interface
   - B) Definir padrões de design de software
   - C) Definir padrões de design de banco de dados
   - D) Definir padrões de design de hardware
   - E) Definir padrões de design de rede

Resposta correta: B) Definir padrões de design de software

2. Qual é o significado da sigla GRASP?
   - A) General Responsibility Assignment Software Patterns
   - B) Graphical Representation of Assignment Software Patterns
   - C) General Representation of Assignment Software Principles
   - D) Graphical Responsibility Assignment Software Principles
   - E) General Responsibility Assignment Software Principles

Resposta correta: A) General Responsibility Assignment Software Patterns

3. Qual é o principal objetivo do padrão GRASP "Creator"?
   - A) Atribuir responsabilidades de criação de objetos
   - B) Atribuir responsabilidades de leitura de objetos
   - C) Atribuir responsabilidades de atualização de objetos
  

### **Criação dos Agentes, vetorialização e RAG**

In [ ]:
# RAG usando RetrievalQA - recuperar informações e gerar questões de múltipla escolha, buscando informações no vetor.
def retrieve_content_using_rag(retriever, content):
    qa_chain = RetrievalQA.from_chain_type(
        llm = llm,
        retriever = retriever, # A vetorialização vai depender do tipo de arquivo recebido
        chain_type = "stuff",
        return_source_documents=False,
    )

    response = qa_chain.run( f"Based on the theme '{content}', retrieve information and generate 5 contextualized questions in with 5 multiple-choice alternatives with answers in Markdown, translate to Brazilian Portuguese.")
    return response

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
# Criação das tools/ferramentas
tools = [
    Tool(
        # Agente 1: Recupera a informação utilizando RAG
        name = "RetrieveContent",
        func = lambda content: retrieve_content_using_rag(retriever, content),
        description = "Searches for relevant information in the vector based on {content} using RAG",
    ),

    Tool(
         # Agente 2: Gera as perguntas baseado nas informações do Agente 1
        name = "GenerateQuestions",
        func = lambda content: llm.predict(
            f"You are an assistant who generates multiple choice questions about the content. Based on the following content using the information of the RetrieveContent agent"
            "Generate 5 multiple choice questions with 5 alternatives each in Portuguese on the topic and provide the answers in Markdown."
        ),
        description = "Generates multiple choice questions with 5 alternatives each in Portuguese based on the content provided.",
    ),
]

In [ ]:
# Inicialização dos agentes
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


### **Extrair informações a partir de uma URL**

In [ ]:
# Carregar e vetorializar o conteúdo da URL
def url_to_vector(url):
    loader = WebBaseLoader(url)
    docs = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # splitar o texto para tornar mais eficiente
    texts = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings() # embeddings
    vector = FAISS.from_documents(texts, embeddings) # vetor de armazenamento

    return vector.as_retriever()

In [ ]:
url = 'https://www.geeksforgeeks.org/kruskals-minimum-spanning-tree-algorithm-greedy-algo-2/'
retriever = url_to_vector(url)

# Usar o agente para responder
content = "Kruskal’s Minimum Spanning Tree (MST) Algorithm"
response = agent.run({"input": f"Recupere informações sobre '{content}' e gere 5 questões relevantes e contextualizadas de múltipla escolha sobre o tema, perguntas e respostas em Markdown.", "content": content})
print(response)



> Entering new AgentExecutor chain...
I should first retrieve information about Kruskal’s Minimum Spanning Tree (MST) Algorithm using RetrieveContent
Action: RetrieveContent
Action Input: Kruskal’s Minimum Spanning Tree (MST) Algorithm
Observation: ### Perguntas sobre o Algoritmo da Árvore de Extensão Mínima de Kruskal

1. Qual é o conceito fundamental abordado pelo algoritmo de Kruskal?
   - A) Ordenação de vértices em um grafo
   - B) Encontrar a Árvore de Extensão Mínima em um grafo ponderado
   - C) Identificar ciclos em um grafo
   - D) Realizar a busca em largura em um grafo
   - E) Calcular a distância entre dois vértices em um grafo

**Resposta Correta:** B) Encontrar a Árvore de Extensão Mínima em um grafo ponderado

2. Qual é a abordagem principal do algoritmo de Kruskal para encontrar a Árvore de Extensão Mínima?
   - A) Abordagem Aleatória
   - B) Abordagem de Força Bruta
   - C) Abordagem Gulosa
   - D) Abordagem de Divisão e Conquista
   - E) Abordagem Heurística

**Res

In [ ]:
url = 'https://pt.wikipedia.org/wiki/Padrão_de_projeto_de_software'
retriever = url_to_vector(url)

# Usar o agente para responder
content = "Padrões de projeto de Software"
response = agent.run({"input": f"Recupere informações sobre '{content}' e gere 5 questões relevantes e contextualizadas de múltipla escolha sobre o tema, perguntas e respostas em Markdown.", "content": content})
print(response)



> Entering new AgentExecutor chain...
I should first retrieve information about 'Padrões de projeto de Software' using the RetrieveContent tool.
Action: RetrieveContent
Action Input: 'Padrões de projeto de Software'
Observation: ### Padrões de Projeto de Software

1. Qual é a definição de Padrões de Projeto de Software?
   - A) São modelos de design exclusivos para cada projeto.
   - B) São soluções reutilizáveis para problemas comuns no desenvolvimento de software.
   - C) São algoritmos complexos para otimizar o código.
   - D) São métodos de programação específicos para linguagens de baixo nível.
   - E) São técnicas de depuração de software.

**Resposta correta: B) São soluções reutilizáveis para problemas comuns no desenvolvimento de software.**

2. Quais são os Padrões GoF ('Gang of Four') mencionados no contexto?
   - A) Padrões de criação, padrões estruturais e padrões comportamentais.
   - B) Padrões de segurança, padrões de desempenho e padrões de interface.
   - C) Padrões

### **Extrair informações a partir de um pdf**

In [ ]:
# Carregar e vetorializar o conteúdo do pdf
def pdf_to_vector(file_path):
    loader = PyPDFLoader(file_path)
    docs = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(docs) # splitar o arquivo para melhorar a eficiência

    embeddings = OpenAIEmbeddings() # embeddings
    vector = FAISS.from_documents(texts, embeddings) # vetor de armazenamento

    return vector.as_retriever()

In [ ]:
file_path = '/content/Estruturas de Dados e Algoritmos.pdf'
retriever = pdf_to_vector(file_path)

# Usar o agente para responder
content = "Análise Assintótica"
response = agent.run({"input": f"Recupere informações sobre '{content}' e gere 5 questões relevantes e contextualizadas de múltipla escolha sobre o tema, perguntas e respostas em Markdown.", "content": content})
print(response)



> Entering new AgentExecutor chain...
I should first retrieve information about 'Análise Assintótica' using RetrieveContent.
Action: RetrieveContent
Action Input: 'Análise Assintótica'
Observation: 1. Qual é o objetivo da análise assintótica na avaliação de algoritmos?
   - A) Considerar apenas entradas pequenas
   - B) Ignorar constantes e expoentes de menor magnitude
   - C) Comparar funções de tempo de execução de forma complexa
   - D) Determinar a ordem exata de crescimento das funções
   - E) Reduzir a eficiência dos algoritmos

Resposta: B) Ignorar constantes e expoentes de menor magnitude

2. Por que a análise assintótica é utilizada na comparação entre funções?
   - A) Para aumentar a complexidade das funções
   - B) Para considerar apenas constantes
   - C) Para simplificar a comparação entre funções
   - D) Para diminuir a ordem de crescimento das funções
   - E) Para tornar as funções menos eficientes

Resposta: C) Para simplificar a comparação entre funções

3. Qual é a 

In [ ]:
file_path = '/content/grafosdirecionados.pdf'
retriever = pdf_to_vector(file_path)

# Usar o agente para responder
content = "Grafos Direcionados"
response = agent.run({"input": f"Recupere informações sobre '{content}' e gere 5 questões relevantes e contextualizadas de múltipla escolha sobre o tema, perguntas e respostas em Markdown.", "content": content})
print(response)



> Entering new AgentExecutor chain...
I should first retrieve information about 'Grafos Direcionados' using the RetrieveContent tool.
Action: RetrieveContent
Action Input: 'Grafos Direcionados'
Observation: ### Perguntas sobre Grafos Direcionados

1. Qual é a definição de um grafo direcionado?
   - A) Um conjunto vazio de objetos.
   - B) Um conjunto de vértices não ordenados.
   - C) Um conjunto de arestas sem direção.
   - D) Um conjunto de vértices e arestas, com uma aplicação que associa cada aresta a um par ordenado de vértices.
   - E) Um conjunto de vértices representados por linhas.

2. Por que é importante associar sentido às arestas de um grafo?
   - A) Para tornar o grafo mais bonito visualmente.
   - B) Para facilitar a contagem de vértices.
   - C) Para representar corretamente situações práticas como ruas de mão única ou sequência de execução de instruções.
   - D) Para diminuir a complexidade do grafo.
   - E) Para adicionar mais vértices ao grafo.

3. Como são represe

### **Extrair informações a partir de um aqrquivo .txt**

In [ ]:
# Carregar o vetorializar o arquivo .txt
def txt_to_vector(file_path):
    loader = TextLoader(file_path, encoding="utf-8")
    docs = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(docs) # splitar

    embeddings = OpenAIEmbeddings() # embeddings
    vector = FAISS.from_documents(texts, embeddings) #vetor de armazenamento

    return vector.as_retriever()

In [ ]:
file_path = '/content/probabilidade_1.txt'
retriever = txt_to_vector(file_path)

# Usar o agente para responder
content = "Introdução à Probabilidade"
response = agent.run({"input": f"Recupere informações sobre '{content}' e gere 5 questões relevantes e contextualizadas de múltipla escolha sobre o tema, perguntas e respostas em Markdown.", "content": content})
print(response)



> Entering new AgentExecutor chain...
I should start by retrieving information on 'Introdução à Probabilidade' using the RetrieveContent tool.
Action: RetrieveContent
Action Input: Introdução à Probabilidade
Observation: ### Perguntas sobre Introdução à Probabilidade

1. Qual é a definição de probabilidade condicional?
   - A) A probabilidade de um evento ocorrer independentemente de outro.
   - B) A probabilidade de um evento ocorrer dado que outro evento já ocorreu.
   - C) A probabilidade de um evento nunca ocorrer.
   - D) A probabilidade de um evento ocorrer em qualquer situação.
   - E) A probabilidade de um evento ocorrer em um futuro distante.

**Resposta: B) A probabilidade de um evento ocorrer dado que outro evento já ocorreu.**

2. Como a probabilidade pode ser aproximada pela frequência relativa?
   - A) Através de cálculos complexos.
   - B) Através de observações repetidas do fenômeno aleatório.
   - C) Através de previsões matemáticas.
   - D) Através de estimativas su